In [ ]:
import sys
sys.path.insert(1, '../Modules/')
from chunck import Chunck
from segmentator import Segmentator

In [ ]:
seg = Segmentator('../Models/vosk-model-small-ru-0.4/', 
                 '../Models/shape_predictor_68_face_landmarks.dat', 1)

In [ ]:
seg.segment_to_chuncks('../Parsing/Боль и страдания Первый месяц  ШАД ЯНДЕКСА.mp4',
                       'TestDataSet/')

In [ ]:
import os
import sys
import subprocess
import json
import math

import cv2
import vosk
import librosa
import pytube as pt
import numpy as np
import pandas as pd
import moviepy.editor as mp

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

sys.path.insert(1, '../Modules/')
from chunck import Chunck

In [ ]:
class Segmentator:
	"""Класс для сегментации видоефайлов"""
	def __init__(self, audio_model_path, landmarks_model_path, conf=1):
		self.landmarks_model_path = landmarks_model_path
		self.audio_model_path = audio_model_path
		self.conf = conf # Вероятность, начиная с которой надо оставлять распознанные слов
		self.word_dict = None
		

	def segment_to_chuncks(self, path, out_path):
		"""
		Метод для сегментации видеофайла по словам в датасет

		path: путь для видео
		out_path: путь для датасета
		"""
		if self.word_dict is None:
			self.word_dict = self.segment_words(path)

		for key in self.word_dict:
			for index, word in enumerate(self.word_dict[key]):
				max_index = 0

				if word[2] >= self.conf:
					l_path = f'{out_path}/{word[0]}'
					w_path = f'{out_path}/{word[0]}/{word}'

					if not os.path.exists(l_path):
						os.mkdir(l_path)
					if not os.path.exists(w_path):
						os.mkdir(w_path)
					else:

						files = os.listdir()
						for file in files:
							max_index = max(max_index, file.partition('.'))

					temp_path = f'{w_path}/{max_index}t.mp4'
					ffmpeg_extract_subclip(path, word[0], word[1], temp_path)

					ch = Chunck(path, self.landmarks_model_path)
					ch.prepare()
					ch.to_file(f'{w_path}/{max_index}.mp4')

					if os.path.isfile(temp_path):
						os.remove(temp_path)
						

	def _extract_words(self, res):
		jres = json.loads(res)
		if not 'result' in jres:
			return []
		words = jres['result']
		return words

	def _transcribe_words(self, recognizer, bytes):
		result = []

		chunck_size = 4000
		for chunck_no in range(math.ceil(len(bytes)) / chunck_size):
			start = chunck_size * chunck_no
			end = min(len(bytes), (chunck_no + 1) * chunck_size)
			data = bytes[start : end]

			if recognizer.AcceptWaveform(data):
				words = self._extract_words(recognizer.Result())
				result += words

		result += self._extract_words(recognizer.FinalResult())

		return result

	def segment_words(self, input_path, out_path=None):
		"""
		Метод для сбора информации о таймингах слов в
		словарь или cvs файл

		input_path: путь к видеофайлу
		out_path: путь к csv файлу. Если None, то сохранять не надо

		returns: словарь слов с таймингами и вероятностями
		"""

		clip = mp.VideoFileClip(input_path)
		temp_audio_path = 'audio.mp3'
		clip.audio.write_audiofile(temp_audio_path)

		vosk.SetLogLevel(-1)

		sample_rate = 16000
		audio, sr = librosa.load(temp_audio_path, sr=sample_rate)

		int16 = np.int16(audio * 32768).tobytes()

		model = vosk.Model(self.audio_model_path)
		recognizer = vosk.KaldiRecognizer(model, sample_rate)

		res = transcribe_words(recognizer, int16)
		df = pd.DataFrame.from_records(res)
		df = df.sort_values('start')

		if out_path is not None:
			df.to_cvs(out_path, index=False)


		word_dict = {}
		for index, row in df.iterrows():
			word_dict[row['word']] = []

		for index, row in df.iterrows():
			word_dict[row['word']].append((row['start'], row['end'], row['conf']))

		if os.path.isfile(temp_audio_path):
			os.delete(temp_audio_path)

		return word_dict

In [ ]:
s = Segmentator('../Models/vosk-model-small-ru-0.4/',
               '../Models/shape_predictor_68_face_landmarks.dat')

In [ ]:
s.segment_to_chuncks('../Parsing/Боль и страдания Первый месяц  ШАД ЯНДЕКСА.mp4', 
                     'TestDataSet/')

In [ ]:
s.segment_words('../Parsing/Боль и страдания Первый месяц  ШАД ЯНДЕКСА.mp4')

In [ ]:
import sys
import os
import subprocess
import json
import math

import vosk
import librosa
import numpy as np
import pandas as pd
import moviepy.editor as mp

sys.path.insert(1, '../Modules/')
from chunck import Chunck
from segmentator import Segmentator

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [ ]:
class Segmentator:
    def __init__(self, audio_model_path, video_model_path,
                conf=1):
        self.audio_model_path = audio_model_path
        self.video_model_path = video_model_path
        self.conf = conf
        
    def _to_dict(self, df):
        result = {}
        for index, row in df.iterrows():
            if row['conf'] >= self.conf:
                result[row['word']] = []

        for index, row in df.iterrows():
            if row['conf'] >= self.conf:
                result[row['word']].append((row['start'], row['end']))
        return result
        
    def _extract_words(self, res):
        jres = json.loads(res)
        if not 'result' in jres:
            return []
        words = jres['result']
        return words
    
    def _transcribe_words(self, recognizer, bytes):
        result = []

        chunk_size = 4000
        for chunk_no in range(math.ceil(len(bytes) / chunk_size)):
            start = chunk_no * chunk_size
            end = min(len(bytes), (chunk_no + 1) * chunk_size)
            data = bytes[start : end]

            if recognizer.AcceptWaveform(data):
                words = self._extract_words(recognizer.Result())
                result += words
        result += self._extract_words(recognizer.FinalResult())

        return result
    
    def get_info(self, input_path, out_path=None):
        vosk.SetLogLevel(-1)
        
        temp_audio_path = 'temp.wav'
        clip = mp.VideoFileClip(input_path)
        clip.audio.write_audiofile(temp_audio_path)

        sample_rate = 16000
        audio, sr = librosa.load(temp_audio_path, sr=sample_rate)

        int16 = np.int16(audio * 32768).tobytes()

        model = vosk.Model(self.audio_model_path)
        recognizer = vosk.KaldiRecognizer(model, sample_rate)

        res = self._transcribe_words(recognizer, int16)
        df = pd.DataFrame.from_records(res)
        df = df.sort_values('start')
        
        if os.path.isfile(temp_audio_path):
            os.remove(temp_audio_path)
        
        if out_path is not None:
            df.to_csv(out_path, index=False)
            
        return self._to_dict(df)
    
    def get_chuncks(self, input_path, out_path='', size=(50, 50),
                    word_dict=None):
        # Create a dict for segmented words timestamps
        if word_dict is None:
            word_dict = self.get_info(input_path)
        
        if not os.path.exists(out_path):
            os.mkdir(out_path)
            
        for word in word_dict:
            w_num = 0
            l_path = f'{out_path}/{word[0]}'
            w_path = f'{l_path}/{word}'
            
            if not os.path.exists(l_path):
                os.mkdir(l_path)
                
            if not os.path.exists(w_path):
                os.mkdir(w_path)
              
            filenames = os.listdir(w_path)
            for filename in filenames:
                string_num = filename.partition('.')[0]
                if string_num.isdigit():
                    w_num = max(w_num, int(filename.partition('.')[0]))
                
            for current_word in word_dict[word]:
                print(current_word)
                temp_video_path = f'{w_path}/temp.mp4'

                ffmpeg_extract_subclip(input_path, current_word[0], 
                                       current_word[1],
                                       targetname=temp_video_path)
                
                ch = Chunck(temp_video_path, self.video_model_path, size)
                if not ch.prepare():
                    continue
                    
                try:
                    ch.to_file(f'{w_path}/{w_num}.avi')
                except ZeroDivisionError:
                    continue
                
                df = pd.DataFrame(columns=['length'],
                                 data=[current_word[1] - current_word[0]])
                df.to_csv(f'{w_path}/{w_num}.csv', index=False)
                
                w_num += 1
                
                if os.path.isfile(temp_video_path):
                    os.remove(temp_video_path)
                    
                    
        pass
    

In [ ]:
s = Segmentator('../Models/vosk-model-small-ru-0.4/', 
                '../Models/shape_predictor_68_face_landmarks.dat')

In [ ]:
d = s.get_info('../Parsing/Боль и страдания Первый месяц  ШАД ЯНДЕКСА.mp4')

In [ ]:
s.get_chuncks('../Parsing/Боль и страдания Первый месяц  ШАД ЯНДЕКСА.mp4', 
              'TestDataSet/')

In [ ]:
ch = Chunck('../Parsing/Audio/Dataset/з/забавно/0.mp4',
            '../Models/shape_predictor_68_face_landmarks.dat')

In [ ]:
ch.prepare()

In [ ]:
ch.show(0.5)

In [ ]:
print(ch.x_max_landmark, ch.x_min_landmark)

In [ ]:
ch.to_file('test1.avi')

In [ ]:
import sys
sys.path.insert(1, '../Modules/')
from chunck import Chunck
from segmentator import Segmentator

In [ ]:
s = Segmentator('../Models/shape_predictor_68_face_landmarks.dat',
                '../Models/vosk-model-small-ru-0.4/')


In [ ]:
import cv2
import time
import dlib
import numpy as np

In [ ]:
cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../Models/shape_predictor_68_face_landmarks.dat')
length = 60 - 48
x, prev_x = np.zeros(length), np.zeros(length)
y, prev_y = np.zeros(length), np.zeros(length)
velocity_x, velocity_y = 0, 0
detections = 0

fps = cap.get(cv2.CAP_PROP_FPS)

iteration = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face).parts()[48:60]
        if iteration <= 5:
            detections += 1
        current_velocity_x, current_velocity_y = 0, 0
        for i, landmark in enumerate(landmarks):
            prev_x[i], x[i] = x[i], landmark.x
            prev_y[i], y[i] = y[i], landmark.y
            
            if iteration <= 5:
                current_velocity_x += np.abs(x[i] - prev_x[i])
                current_velocity_y += np.abs(y[i] - prev_y[i])
            
            cv2.circle(frame, (landmark.x, landmark.y),
                      2, (0, 0, 255))
            
        current_velocity_x /= length
        current_velocity_y /= length
        velocity_x += current_velocity_x
        velocity_y += current_velocity_y
        
    print(velocity_x, velocity_y)
        
            
    cv2.imshow('video', frame)
        
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

velocity_x /= detections
velocity_y /= detections

velocity_x *= fps
velocity_y *= fps

print(velocity_x, velocity_y)

In [ ]:
detections

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(columns=['l', 'v1', 'v2'],
                  data=[[2, 4, 5]])

In [ ]:
df = pd.DataFrame(columns=['url'],
                 data=[['asdsfdsf'], ['sdfdsfds'],
                       ['sdfdsfdsfdsf'],
                       ['sdfdsfdsf']])

In [ ]:
n = df.to_numpy()
for el in n:
    print(el[0])

In [15]:
import sys
import os
import pytube as pt
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import pandas as pd
import re

sys.path.insert(1, '../Modules/')
from segmentator import Segmentator

In [86]:
class Parser:
    def __init__(self, seg, report_path, 
                 headers={
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:85.0) Gecko/20100101 Firefox/85.0'
      }):
        self.headers = headers
        self.seg = seg
        self.report_path = report_path
        self.columns = ['word', 'length', 'velocity_x', 'velocity_y', 'author']
        
    def parse(self, url, out_path):
        youtube = pt.YouTube(url)
        temp_video_path = 'temp.mp4'
        video = youtube.streams.first().download(filename='temp')
        partial_df = seg.get_chuncks(temp_video_path, out_path)
        df = pd.DataFrame(columns=[self.columns[-1]],
                          data=[[youtube.author]])
        
        report = pd.concat([partial_df, df], axis=1)
        
        if os.path.exists(temp_video_path):
            os.remove(temp_video_path)
            
        report.to_csv(self.report_path, index=False)
        return report
        
    def from_csv(self, csv_path, out_path):
        urls_df = pd.read_csv(csv_path)
        urls_temp = urls_df.to_numpy()
        urls = []
        for url in urls_temp:
            urls.append(url[0])
        urls = np.array(urls, dtype=str)
        report = pd.DataFrame(columns=self.columns)
        
        for url in urls:
             report = pd.concat([report, self.parse(url, out_path)])
                
        report.to_csv(self.report_path, index=False)
        
        return report
    

In [87]:
seg = Segmentator('../Models/vosk-model-small-ru-0.4/',
                  '../Models/shape_predictor_68_face_landmarks.dat')
p = Parser(seg, 'test_report.csv')

In [88]:
p.parse('https://www.youtube.com/watch?v=GOTMuNTa4kQ',
        'TestParse/')

chunk:  17%|█▋        | 274/1643 [00:00<00:00, 2739.34it/s, now=None]

MoviePy - Writing audio in temp.wav


MoviePy - Done.
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:

,word,length,velocity_x,velocity_y,author
0,меня,0.48,22.000000,52.666667,Психолог Вероника Степанова
0,зовут,0.57,10.666667,29.000000,Психолог Вероника Степанова
0,вероника,1.38,16.193548,32.903226,Психолог Вероника Степанова
0,степанова,1.53,12.914286,41.600000,Психолог Вероника Степанова
0,золотой,0.60,11.666667,15.333333,Психолог Вероника Степанова
...,...,...,...,...,...
0,вот,0.33,14.800000,9.200000,Психолог Вероника Степанова
0,дела,0.54,11.818182,31.272727,Психолог Вероника Степанова
0,сегодня,0.45,13.111111,28.888889,Психолог Вероника Степанова
0,своих,0.24,9.500000,14.000000,Психолог Вероника Степанова


In [ ]:
df1 = pd.DataFrame(columns=['a', 'b'], data=[[1, 2]])
df2 = pd.DataFrame(columns=['a', 'b'], data=[[4, 5]])

In [ ]:
pd.concat([df1, df2], ignore_index=True)

In [ ]:
test_df = pd.DataFrame(columns=['url'], data=[['https://www.youtube.com/watch?v=k4J9WGijj7o'],
                                    ['https://www.youtube.com/watch?v=CLuoNrkO8mg']])

In [ ]:
test_df.to_csv('test.csv', index=False)

In [ ]:
import sys
sys.path.insert(1, '../Modules/')
from segmentator import Segmentator
from parser import Parser

In [ ]:
seg = Segmentator('../Models/vosk-model-small-ru-0.4/',
                  '../Models/shape_predictor_68_face_landmarks.dat')

In [ ]:
p = Parser(seg, 'report.csv')

In [ ]:
p.from_cvs('test.csv', 'TestDataSet3/')

In [47]:
p.get_name('https://www.youtube.com/watch?v=GOTMuNTa4kQ')

0


''

In [49]:
p.response.text

'<!DOCTYPE html><html style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" lang="ru-RU"><head><script nonce="kx3Kru5cVM6R/PGTBfxQyw">var ytcfg={d:function(){return window.yt&&yt.config_||ytcfg.data_||(ytcfg.data_={})},get:function(k,o){return k in ytcfg.d()?ytcfg.d()[k]:o},set:function(){var a=arguments;if(a.length>1)ytcfg.d()[a[0]]=a[1];else for(var k in a[0])ytcfg.d()[k]=a[0][k]}};\nwindow.ytcfg.set(\'EMERGENCY_BASE_URL\', \'\\/error_204?t\\x3djserror\\x26level\\x3dERROR\\x26client.name\\x3d1\\x26client.version\\x3d2.20210412.07.00\');</script><script nonce="kx3Kru5cVM6R/PGTBfxQyw">(function(){window.yterr=window.yterr||true;window.unhandledErrorMessages={};window.unhandledErrorCount=0;\nwindow.onerror=function(msg,url,line,columnNumber,error){var err;if(error)err=error;else{err=new Error;err.stack="";err.message=msg;err.fileName=url;err.lineNumber=line;if(!isNaN(columnNumber))err["columnNumber"]=columnNumber}var message=String(err.message);if(!err.message||message in wind

In [62]:
import lxml

In [65]:
url = 'https://www.youtube.com/watch?v=GOTMuNTa4kQ'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [66]:
items = soup.find_all('yt-formatted-string', 
                      attrs={'class':'style-scope ytd-channel-name'})

b'<!DOCTYPE html><html style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" lang="ru-RU"><head><meta http-equiv="X-UA-C'
b'ompatible" content="IE=edge"/><script nonce="jH6ArSXzgrKza06t+nw4nw">var ytcfg={d:function(){return window.yt&&yt.config_||ytcfg'
b'.data_||(ytcfg.data_={})},get:function(k,o){return k in ytcfg.d()?ytcfg.d()[k]:o},set:function(){var a=arguments;if(a.length>1)y'
b"tcfg.d()[a[0]]=a[1];else for(var k in a[0])ytcfg.d()[k]=a[0][k]}};\nwindow.ytcfg.set('EMERGENCY_BASE_URL', '\\/error_204?t\\x3djser"
b'ror\\x26level\\x3dERROR\\x26client.name\\x3d1\\x26client.version\\x3d2.20210412.07.00\');</script><script nonce="jH6ArSXzgrKza06t+nw4nw'
b'">(function(){window.yterr=window.yterr||true;window.unhandledErrorMessages={};window.unhandledErrorCount=0;\nwindow.onerror=func'
b'tion(msg,url,line,columnNumber,error){var err;if(error)err=error;else{err=new Error;err.stack="";err.message=msg;err.fileName=ur'
b'l;err.lineNumber=line;if(!isNaN(columnNumber))err["columnNumber

In [73]:
youtube = pt.YouTube(url)
video = youtube.streams.first().download()

In [84]:
youtube.author_name

AttributeError: 'YouTube' object has no attribute 'author_name'

In [90]:
urls_df = pd.DataFrame(columns=['url'],
                       data=[['https://www.youtube.com/watch?v=mz-effmtkHc'], 
                             ['https://www.youtube.com/watch?v=GOTMuNTa4kQ']])

In [93]:
urls_df.to_csv('urls.csv', index=False)

In [94]:
seg = Segmentator('../Models/vosk-model-small-ru-0.4/',
                  '../Models/shape_predictor_68_face_landmarks.dat')
p = Parser(seg, 'report2.csv')

In [95]:
p.from_csv('urls.csv', 'TestParse2/')

chunk:  11%|█▏        | 278/2442 [00:00<00:00, 2777.25it/s, now=None]

MoviePy - Writing audio in temp.wav


MoviePy - Done.
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:

chunk:  17%|█▋        | 275/1643 [00:00<00:00, 2745.14it/s, now=None]

MoviePy - Writing audio in temp.wav


MoviePy - Done.
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:

,word,length,velocity_x,velocity_y,author
0,привет,0.48,11.875000,20.625000,Блог Насти Каменских
0,спасибо,0.39,16.927083,47.395833,Блог Насти Каменских
0,что,0.15,16.666667,26.041667,Блог Насти Каменских
0,что,0.24,9.375000,44.270833,Блог Насти Каменских
0,что,0.21,27.083333,22.916667,Блог Насти Каменских
...,...,...,...,...,...
0,вот,0.33,14.800000,9.200000,Психолог Вероника Степанова
0,дела,0.54,11.818182,31.272727,Психолог Вероника Степанова
0,сегодня,0.45,13.111111,28.888889,Психолог Вероника Степанова
0,своих,0.24,9.500000,14.000000,Психолог Вероника Степанова


In [1]:
import sys
sys.path.insert(1, '../Modules/')
from segmentator import Segmentator
from dsparser import Parser

In [2]:
seg = Segmentator('../Models/vosk-model-small-ru-0.4/',
                  '../Models/shape_predictor_68_face_landmarks.dat')
p = Parser(seg, '../../DataSet/Report/report.csv')

In [3]:
p.from_csv('../../DataSet/Report/urls.csv', '../../DataSet/')

chunk:   1%|          | 323/32672 [00:00<00:10, 3223.39it/s, now=None]

MoviePy - Writing audio in temp.wav


MoviePy - Done.
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

...

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

error: OpenCV(4.5.1) /tmp/pip-req-build-jr1ur_cf/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [7]:
p.seg